<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Note" data-toc-modified-id="Note-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Note</a></span></li><li><span><a href="#Load-data-and-the-bs4-package" data-toc-modified-id="Load-data-and-the-bs4-package-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data and the <code>bs4</code> package</a></span></li><li><span><a href="#Parse-data-via-the-page-structure" data-toc-modified-id="Parse-data-via-the-page-structure-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parse data via the page structure</a></span></li><li><span><a href="#Combine-all-lists-to-form-one-data-frame" data-toc-modified-id="Combine-all-lists-to-form-one-data-frame-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combine all lists to form one data frame</a></span></li></ul></div>

# Note
Written by my team member Long Nguyen.

Edited and structured by me. 

# Load data and the `bs4` package

We will be focusing on this IMDB list of the best comedy films of all time.

In [7]:
from requests import get
url = 'https://www.imdb.com/list/ls000729643/'
response = get(url)

In [8]:
from bs4 import BeautifulSoup
raw_html = BeautifulSoup(response.text, 'html.parser')

# Parse data via the page structure

Each movie is identified by a `div` with the class class `list-item mode-detail`.

In [9]:
movies = raw_html.find_all('div', class_ = 'lister-item mode-detail')

Below are all the preprocessing and cleaning operations already done in the R version. <br>However, a notable difference is how `BeautifulSoup` relies on page structure (e.g `movie.h3.find`) to get to the content.

In [10]:
# Lists to store the scraped data in
ranks = []
titles = []
descriptions = []
runtimes = []
genres = []
ratings = []
metascores = []
votes = []
gross_earnings = []
directors = []
actors = []


# Extract data from individual movie container
for movie in movies:

        # The rank
        rank = movie.h3.find('span', class_ = 'text-primary').text 
        ranks.append(rank)
        
        # The name
        title = movie.h3.a.text
        titles.append(title)
        
        # The description
        raw_description = movie.find('p', class_ = '').text
        description = raw_description.replace("\n","")
        descriptions.append(description)
        
        # The runtime
        raw_runtime = movie.p.find('span', class_ = 'runtime').text
        runtime = raw_runtime.replace("min","")
        runtimes.append(int(runtime))
        
        # The genre
        raw_genre = movie.p.find('span', class_ = 'genre').text
        genre = raw_genre.replace("\n","")
        genres.append(genre)
        
        # The IMDB rating
        rating = float(movie.find('span', class_='ipl-rating-star__rating').text)
        ratings.append(rating)

        # The Metascore
        metascore = movie.find('span', class_ = 'metascore').text
        metascores.append(int(metascore))

        # The number of votes
        vote = movie.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))
        
        # The gross earning in millions
        raw_earning = movie.select("span[name=nv]")[1].text
        raw_earning_wo_dollar = raw_earning.replace("$","")
        earning = raw_earning_wo_dollar.replace("M","")
        gross_earnings.append(float(earning)) 
        
        # List directors and actors
        raw_directorandactor = movie.select("p.text-muted.text-small")[1].text
        directorandactor = raw_directorandactor.replace("\n","")
        
            #Separate directors and actors
        director = directorandactor.split('|')[0]
        actor = directorandactor.split('|')[1]
            
            #Data preprocessing 
                #Director
        director_no_space = director.replace('\n','')
        director_final = director_no_space.replace('Director:','')
        director_final = director_final.replace('Directors:','') 
        directors.append(director_final)
                #Actor
        actor_no_space = actor.replace('\n','')
        actor_final = actor.replace('Stars:', '')
        actors.append(actor_final)
        


# Combine all lists to form one data frame

In [11]:
import pandas as pd

movies_df = pd.DataFrame({'rank': ranks,
                        'movie': titles,
                        'description': descriptions,
                        'runtime': runtimes,
                        'genre': genres,
                        'rating': ratings,
                        'metascore': metascores,
                        'votes': votes,
                        'gross earning in millions': gross_earnings,
                        'directors': directors,
                        'actors': actors
                       })

In [12]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
rank                         50 non-null object
movie                        50 non-null object
description                  50 non-null object
runtime                      50 non-null int64
genre                        50 non-null object
rating                       50 non-null float64
metascore                    50 non-null int64
votes                        50 non-null int64
gross earning in millions    50 non-null float64
directors                    50 non-null object
actors                       50 non-null object
dtypes: float64(2), int64(3), object(6)
memory usage: 3.2+ KB


In [13]:
movies_df.head(10)

,rank,movie,description,runtime,genre,rating,metascore,votes,gross earning in millions,directors,actors
0,1.,Wedding Crashers,"John Beckwith and Jeremy Grey, a pair of c...",119,"Comedy, Romance",7.0,64,314206,209.22,David Dobkin,"Owen Wilson, Vince Vaughn, Rachel McAdams..."
1,2.,Anchorman: The Legend of Ron Burgundy,Ron Burgundy is San Diego's top-rated news...,94,Comedy,7.2,63,308168,85.29,Adam McKay,"Will Ferrell, Christina Applegate, Steve ..."
2,3.,Walk Hard: The Dewey Cox Story,Singer Dewey Cox overcomes adversity to be...,96,"Comedy, Music",6.8,63,62649,18.32,Jake Kasdan,"John C. Reilly, Jenna Fischer, David Krum..."
3,4.,Step Brothers,Two aimless middle-aged losers still livin...,98,Comedy,6.9,51,246430,100.47,Adam McKay,"Will Ferrell, John C. Reilly, Mary Steenb..."
4,5.,The Hangover,Three buddies wake up from a bachelor part...,100,Comedy,7.7,73,668365,277.32,Todd Phillips,"Zach Galifianakis, Bradley Cooper, Justin..."
5,6.,Office Space,Three company workers who hate their jobs ...,89,Comedy,7.8,68,226641,10.82,Mike Judge,"Ron Livingston, Jennifer Aniston, David H..."
6,7.,Dumb and Dumber,The cross-country adventures of 2 good-hea...,107,Comedy,7.3,41,329041,127.18,"Peter Farrelly, Bobby Farrelly","Jim Carrey, Jeff Daniels, Lauren Holly, M..."
7,8.,Austin Powers: International Man of Mystery,A 1960s secret agent is brought out of cry...,89,"Adventure, Comedy",7.0,51,206931,53.88,Jay Roach,"Mike Myers, Elizabeth Hurley, Michael Yor..."
8,9.,Austin Powers: The Spy Who Shagged Me,Dr. Evil is back and has invented a new ti...,95,"Action, Adventure, Comedy",6.6,59,203704,206.04,Jay Roach,"Mike Myers, Heather Graham, Michael York,..."
9,10.,Starsky & Hutch,Two streetwise cops bust criminals in thei...,101,"Comedy, Crime",6.1,55,131449,88.24,Todd Phillips,"Ben Stiller, Owen Wilson, Snoop Dogg, Vin..."


**Now we have a well-structured dataframe ready to be exported and analyzed**